In [ ]:
%pip install tensorflow pandas matplotlib scikit-learn

In [ ]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np

In [ ]:
df = pd.read_csv('train.csv')

# Preprocessing

In [ ]:
from tensorflow.keras.layers import TextVectorization

In [ ]:
df.columns

In [ ]:
df[df.columns[2:]]

In [ ]:
X = df["comment_text"]
Y = df[df.columns[2:]].values

In [ ]:
MAX_FEATURES = 200000

In [ ]:
vectorizer = TextVectorization(max_tokens=MAX_FEATURES, output_sequence_length=1800, output_mode='int')

In [ ]:
vectorizer.adapt(X.values)

In [ ]:
vectorizer.get_vocabulary()

In [ ]:
vectorized_text = vectorizer(X.values)
vectorized_text

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text,Y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8)

In [ ]:
train = dataset.take(int(len(dataset)*.7))
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

# Using a Sequential Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding

In [ ]:
model = Sequential()
model.add(Embedding(MAX_FEATURES+1,32))
model.add(Bidirectional(LSTM(32,activation='tanh')))
model.add(Dense(128,activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(6,activation='sigmoid'))

In [ ]:
model.compile(loss='BinaryCrossentropy',optimizer='Adam')

In [ ]:
model.summary()

In [ ]:
history = model.fit(train,validation_data=val,epochs=5)

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
plt.figure(figsize=(8,5))
pd.DataFrame(history.history).plot()
plt.show()

# Predictions

In [ ]:
input_text = vectorizer('You suck')

In [ ]:
res = model.predict(input_text)

In [ ]:
(res>0.5).astype(int)

In [ ]:
batch_X, batch_Y = test.as_numpy_iterator().next()

In [ ]:
(model.predict(batch_X)>0.5).astype(int)

In [ ]:
res.shape

# Evaluation

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy

In [ ]:
pre = Precision()
rec = Recall()
acc = CategoricalAccuracy()

In [ ]:
for batch in test.as_numpy_iterator(): 
    # Unpack the batch 
    X_true, Y_true = batch
    # Make a prediction 
    Yhat = model.predict(X_true)
    
    # Flatten the predictions
    Y_true = Y_true.flatten()
    yhat = yhat.flatten()
    
    pre.update_state(Y_true, yhat)
    rec.update_state(Y_true, yhat)
    acc.update_state(Y_true, yhat)


In [ ]:
print(f'Precision: {pre.result().numpy()}, Recall:{rec.result().numpy()}, Accuracy:{acc.result().numpy()}')

# Gradio

In [ ]:
%pip install gradio jinja2


In [ ]:
import tensorflow as tf
import gradio as gr

In [ ]:
model.save('toxicity')

In [ ]:
model = tf.keras.models.load_model('toxicity')

In [ ]:
input_str = vectorizer('hey i freaken hate you!')

In [ ]:
res = model.predict(np.expand_dims(input_str,0))
res

In [ ]:
def score_comment(comment):
    vectorized_comment = vectorizer([comment])
    results = model.predict(vectorized_comment)
    
    text = ''
    for idx, col in enumerate(df.columns[2:]):
        text += '{}: {}\n'.format(col, results[0][idx]>0.5)
    
    return text

In [ ]:
interface = gr.Interface(
    fn=score_comment, 
    inputs=gr.inputs.Textbox(lines=2, placeholder='Comment to score'),
    outputs='text'
)


In [ ]:
interface.launch(share=True)